In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gender-submission-csv/gender_submission.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic-machine-learning-from-disaster/train.csv
/kaggle/input/titanic-machine-learning-from-disaster/test.csv


# **Step 1: Load and View the Data



In [2]:
# Step 1: Import libraries
import pandas as pd
import numpy as np

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the data
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

# View first few rows
train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# **#  Step 2: Explore Missing Values**

In [4]:
# Check for missing values
train_df.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**#  Step 3: Handle Missing Data**

- We removed the `Cabin` column because it has many missing values (if it exists).
- We filled missing values in `Age` with the **median**.
- We filled missing `Embarked` values in the training set with the **mode**.
- We filled the missing value in `Fare` in the test set using the **median**.


In [5]:
# Safely drop the 'Cabin' column (only if it exists)
train_df = train_df.drop(columns=['Cabin'], errors='ignore')
test_df = test_df.drop(columns=['Cabin'], errors='ignore')

# Fill 'Age' with median
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median())

# Fill 'Embarked' with mode (most frequent value)
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])

# Fill 'Fare' in test set with median
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].median())


# **Step 4: Remove Outliers**

Let’s remove outliers from Fare and Age using IQR (interquartile range).

In [6]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]

# Apply to 'Fare' and 'Age'
train_df = remove_outliers(train_df, 'Fare')
train_df = remove_outliers(train_df, 'Age')


# **Step 5: Save Cleaned Data**

In [7]:
train_df.to_csv('/kaggle/working/cleaned_train.csv', index=False)
test_df.to_csv('/kaggle/working/cleaned_test.csv', index=False)
